In [1]:
%run run_model.py --process=0 --rawname=1 --resume=0 --anchoring=0

24-02-07 00:02:43|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train + Test + Instance
--Start Training New!
--Model_name is set to TRA_LSTM_day_ShortTest!
use /home/mengkjin/Workspace/learndl/scripts/util/../../data/torch_pack/day+rtn11+res11.20231220.pt


24-02-07 00:02:46|MOD:run_model   |: Finish Process [Load Data]! Cost 3.3Secs
24-02-07 00:02:46|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [day] : [endpoint_division(True) , history_standardize(True)]


TRA_LSTM_day_ShortTest #0 @20170103 LoadData Cost    2.2Secs
FirstBite Ep#  0 : loss  1.01383, train 0.00195, valid 0.00583, max 0.0058, best 0.0058, lr1.0e-07
FirstBite Ep#  5 : loss  0.88872, train 0.13751, valid 0.13482, max 0.1348, best 0.1348, lr3.8e-03
FirstBite Ep# 10 : loss  0.84957, train 0.18195, valid 0.16235, max 0.1624, best 0.1624, lr1.3e-03
FirstBite Ep# 15 : loss  0.83003, train 0.20415, valid 0.17960, max 0.1796, best 0.1796, lr6.3e-04
24-02-07 00:03:04|MOD:run_model   |: TRA_LSTM_day_ShortTest #0 @20170103|Round 0 FirstBite Ep# 19 Max Epoch|Train 0.2086 Valid 0.1816 BestVal 0.1817|Cost  0.3Min,  0.8Sec/Ep


tensor([[0.3343, 0.3150, 0.3507],
        [0.3387, 0.3141, 0.3472],
        [0.3437, 0.3135, 0.3428],
        ...,
        [0.3255, 0.3390, 0.3354],
        [0.3262, 0.3360, 0.3378],
        [0.3284, 0.3417, 0.3299]], device='cuda:0')


TRA_LSTM_day_ShortTest #0 @20170704 LoadData Cost    1.8Secs
FirstBite Ep#  0 : loss  1.00990, train 0.00577, valid 0.00753, max 0.0075, best 0.0075, lr1.0e-07
FirstBite Ep#  5 : loss  0.87726, train 0.15051, valid 0.13463, max 0.1346, best 0.1346, lr3.8e-03
FirstBite Ep# 10 : loss  0.84361, train 0.18879, valid 0.16744, max 0.1674, best 0.1674, lr1.3e-03
FirstBite Ep# 15 : loss  0.82668, train 0.20790, valid 0.17836, max 0.1839, best 0.1839, lr6.3e-04
24-02-07 00:03:21|MOD:run_model   |: TRA_LSTM_day_ShortTest #0 @20170704|Round 0 FirstBite Ep# 19 Max Epoch|Train 0.2119 Valid 0.1859 BestVal 0.1862|Cost  0.3Min,  0.8Sec/Ep
24-02-07 00:03:22|MOD:run_model   |: Finish Process [Train Model]! Cost 0.0 Hours, 0.3 Min/model, 0.9 Sec/Epoch


tensor([[0.3304, 0.3372, 0.3324],
        [0.3283, 0.3343, 0.3374],
        [0.3253, 0.3336, 0.3411],
        ...,
        [0.3321, 0.3339, 0.3340],
        [0.3318, 0.3303, 0.3380],
        [0.3364, 0.3367, 0.3269]], device='cuda:0')


24-02-07 00:03:22|MOD:run_model   |: Start Process [Test Model]!
24-02-07 00:03:22|MOD:run_model   |: Each Model Date Testing Mean Score({'train': 'pearson', 'valid': 'pearson', 'test': 'pearson'}):
Models            0       0       0
Output         best swalast swabest
20170103     0.1067  0.0890  0.1070
20170704     0.1069  0.0951  0.1053
AllTimeAvg   0.1068  0.0918  0.1062
AllTimeSum    23.82   20.48   23.69
Std          0.0778  0.0730  0.0777
TValue        20.49   18.77   20.42
AnnIR        6.7232  6.1586  6.7000
24-02-07 00:03:39|MOD:run_model   |: Finish Process [Test Model]! Cost 17.3 Secs
24-02-07 00:03:39|MOD:run_model   |: Will not copy to instance!
